In [2]:
import json
from openai import OpenAI
import pandas as pd
from IPython.display import Image, display
import hashlib
import time

client = OpenAI(api_key="sk-proj-sScFMZr4GK3XaQ5mcYFYT3BlbkFJ1orIjm1gKSNT0q30KBMZ")

In [2]:
resumes = pd.read_csv('100K_data/250_resumes.csv')
postings = pd.read_csv('100K_data/400_postings.csv')
display(postings)
display(resumes)

,description
0,Job descriptionA leading real estate firm in N...
1,"At Aspen Therapy and Wellness , we are committ..."
2,The National Exemplar is accepting application...
3,Senior Associate Attorney - Elder Law / Trusts...
4,Looking for HVAC service tech with experience ...
...,...
395,Fundraising Events CoordinatorReports to: Deve...
396,Job Title: Bilingual Legal Assistant/Paralegal...
397,Exciting Opportunity Alert! Discover your next...
398,Company Description\n SEOGidi is a Search Engi...


,Resume,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR
3,HR SPECIALIST Summary Dedica...,HR
4,HR MANAGER Skill Highlights ...,HR
...,...,...
245,"Sure, I'd be happy to help you create a profes...",NaN
246,"Sure, I'd be happy to help you create a profes...",NaN
247,"Sure, I'd be happy to help you create a profes...",NaN
248,"Sure, I'd be happy to help you create a profes...",NaN


In [3]:
system_prompt="""
Your goal is to use someone's resume to tell me if they are qualified for a job.
You will be provided with a resume and a job description. Your output will be a json object containing only one of five categorizations: "not qualified", "minimally qualified", "somewhat qualified", "qualified", "very qualified".
{
	categorization: string // Level of qualification
} 
"""

cache = {}
with open('10K_cache.json', 'r') as json_file:
    cache = json.load(json_file)
with open('100K_data/3.5K_partial_cache.json') as json_file:
	cache.update(json.load(json_file))
with open('100K_data/53K_partial_cache.json') as json_file:
	cache.update(json.load(json_file))


# Returns Task or None if task already exists in the cache
def CreateTask(resume: str, job_description: str) -> dict | None:
	user_prompt = f"Here is my resume:{resume}, and here is the job description: {job_description}"
	key = hashlib.md5(f"{resume}{job_description}".encode()).hexdigest()

	if key in cache:
		return None

	task = {
        "custom_id": f"{key}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-3.5-turbo",
            "temperature": 0.1,
            "response_format": { "type": "json_object" },
            "messages": [
                { "role": "system", "content": system_prompt },
                { "role": "user", "content": user_prompt }
			]
		}
	}
	return task

def WaitForJobToFinish(batch_job_id):
	"""
	Waits for the batch job to finish by polling every 1 minute.

	Parameters:
	- batch_job_id: The ID of the batch job to wait for.

	Returns:
	- The final status of the batch job.
	"""
	while True:
		batch_job = client.batches.retrieve(batch_job_id)

		# Check the status of the batch job
		status = batch_job.status
		print(f"Current status: {status}")

		if status in ['completed', 'failed']:
			return status
        
		# Wait for the specified interval before polling again
		time.sleep(60)

def TasksToCache(tasks, batch_file_name, result_file_name, cache_file_name):

	# Create Batch file
	with open(batch_file_name, 'w') as file:
		for obj in tasks:
			file.write(json.dumps(obj) + '\n')

	# Upload the Batch file to OpenAI
	batch_file = client.files.create(
		file=open(batch_file_name, "rb"),
		purpose="batch"
	)
	print(batch_file)
	
	# Creating the batch job:
	batch_job = client.batches.create(
		input_file_id=batch_file.id,
		endpoint="/v1/chat/completions",
		completion_window="24h"
	)

	status = WaitForJobToFinish(batch_job.id)

	if (status == 'failed'):
		batch_job = client.batches.retrieve(batch_job.id)
		print('batch job failed!')
		print(batch_job)
		return
	
	# Retrieving results
	batch_job = client.batches.retrieve(batch_job.id)
	result_file_id = batch_job.output_file_id
	result = client.files.content(result_file_id).content

	result_file_name

	with open(result_file_name, 'wb') as file:
		file.write(result)

	# Loading data from saved file
	results = []
	with open(result_file_name, 'r') as file:
		for line in file:
			# Parsing the JSON string into a dict and appending to the list of results
			json_object = json.loads(line.strip())
			results.append(json_object)
	# Create a cache from the results
	cache = {}
	for res in results:
		key = res['custom_id']
		result_str = res['response']['body']['choices'][0]['message']['content']
		try:
			result_obj = json.loads(result_str)
			result = result_obj['categorization']
			if result in ("qualified", "very qualified"):
				cache[key] = 1
			else:
				cache[key] = 0
		except Exception as e:
			print(f'GPT returned invalid object for key - {key}. This will need to be manually added')
			print(e)

	with open(cache_file_name, 'w') as cache_file:
		json.dump(cache, cache_file)

def AutoCacheFiller():
	tasks = []
	for job_description in postings['description']:
		for resume_text in resumes['Resume']:
			tasks.append(CreateTask(resume_text, job_description))
	
	total_tasks = len(tasks)
	# Filter out all None values
	tasks = [task for task in tasks if task is not None]
	new_tasks = len(tasks)
	print(f"Found {total_tasks - new_tasks} tasks already in the cache and removed them.")

	batch_size = 100
	split_tasks = [tasks[i:i+batch_size] for i in range(0, len(tasks), batch_size)]

	# Print the number of batches created
	print(f"Number of batches: {len(split_tasks)}")

	i = 543
	for batch in split_tasks:
		TasksToCache(batch, f"100K_auto_data/{i}_batch.jsonl", f"100K_auto_data/{i}_result.jsonl", f"100K_auto_data/{i}_test_cache.jsonl")
		i+=1

AutoCacheFiller()

Found 67894 tasks already in the cache and removed them.
Number of batches: 322
FileObject(id='file-m1t80UlhU5dffUyFX0orxV0b', bytes=545753, created_at=1721259215, filename='543_batch.jsonl', object='file', purpose='batch', status='processed', status_details=None)
Current status: validating
Current status: completed
FileObject(id='file-fvh9G5DXV4wNCuEjwCAsEMRa', bytes=1022382, created_at=1721259278, filename='544_batch.jsonl', object='file', purpose='batch', status='processed', status_details=None)
Current status: validating
Current status: completed
FileObject(id='file-CzkJsoUM1M9P1ZdILTiW6SLv', bytes=962272, created_at=1721259340, filename='545_batch.jsonl', object='file', purpose='batch', status='processed', status_details=None)
Current status: in_progress
Current status: completed
FileObject(id='file-T9mhoeX4K5eultOXRo4aJDUt', bytes=937502, created_at=1721259407, filename='546_batch.jsonl', object='file', purpose='batch', status='processed', status_details=None)
Current status: val

In [3]:
# load and save cache

cache = {}
with open('10K_cache.json', 'r') as json_file:
    cache = json.load(json_file)
with open('100K_data/3.5K_partial_cache.json') as json_file:
	cache.update(json.load(json_file))
with open('100K_data/53K_partial_cache.json') as json_file:
	cache.update(json.load(json_file))
# This is a single request that did not go through...
cache.update({'3c3c966726e6cb3a33926eaf15cf9c94' : 1})

directory = '100K_auto_data'

# we already loaded the cache before so now we can just update it
for i in range(0, 865):
	with open(f'100K_auto_data/{i}_test_cache.jsonl') as file:
		file_cache = json.loads(file.read())
		cache.update(file_cache)

output_file = '100K_cache.json'
with open(output_file, 'w') as file:
	json.dump(cache, file)


In [4]:
len(cache)

100000